In [1]:
import pandas as pd
import sys
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os
import requests
from google_maps_reviews import ReviewsClient
api_client = ReviewsClient(api_key=os.getenv("OUTSCRAPE_NEW_KEY"))

In [2]:
data = pd.read_csv("Data/companies_with_address.csv" , delimiter=";")


In [3]:
def get_place_id(name, address, place):
    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        "input": f"{name}, {address} {place}",
        "inputtype": "textquery",
        "fields": "place_id",
        "key": os.getenv("GOOGLE_API_KEY")
    }

    response = requests.get(base_url, params=params)
    result = response.json()

    if result.get("candidates"):
        return result["candidates"][0]["place_id"]
    else:
        return None

In [4]:
data['place_id'] = data.apply(lambda row: get_place_id(row['Naam'], row['Adres'], row['Gemeente']), axis=1)
data['place_id'].isna().sum(), data['place_id'].notna().sum()


(np.int64(215), np.int64(1232))

In [7]:
# save the data with place_id
data = data.dropna(subset=['place_id'])
data.to_csv("Data/companies_with_place_id.csv", index=False, sep=";")

In [5]:
data['reviews'] = data.apply(lambda row: api_client.get_reviews(row['place_id'], 250, language='en') if pd.notna(row['place_id']) and row['place_id'] else None, axis=1)

data




KeyboardInterrupt



In [ ]:
# dataset to csv
data.to_csv("Data/companies_with_reviews.csv", index=False, sep=";")